In [56]:
import time
from threading import Thread
import telebot
from telebot import types
from datetime import datetime
import re
import os


i = 2

In [75]:
def get_data(string):
    res = string.split( )
    res1 = res[1].split(".")
    return [string, '/sd ' + "/".join(res1) + ' /st ' + res[0]]

print(get_data("10:11 10.20.1020"))


def update_bat(task):
#    with open ('call.bat', 'r') as f:
#        old_data = f.read()
#    new_data = old_data.split()
    new_string = "python C:\\Users\Sofa\Documents\GitHub\SPBGU-Python-2021\homeworks\Rebrilova\\13.12.21\phone.py " + task
    with open ('call.bat', 'w') as f:
        f.write(new_string)

update_bat("1 22")

['10:11 10.20.1020', '/sd 10/20/1020 /st 10:11']


In [ ]:
bot = telebot.TeleBot('5009014734:AAHhNUBh9h87W9UAOtTfy0oeJcCvKvlMNig')

users = {}
phones = {}
tasks = {}

@bot.message_handler(commands=['start', 'help'])
def handle_start_help(message):
    keyboard = types.InlineKeyboardMarkup()
    start_button = types.InlineKeyboardButton(text='Начать', callback_data='LetsGo')
    keyboard.add(start_button)
    bot.send_message(message.chat.id, f'''Привет, {str(message.chat.first_name)}! 
                                          Нажми Начать''', reply_markup=keyboard)
@bot.callback_query_handler(func=lambda  call: True)
def callback_worker(call):
    if call.data == 'LetsGo':
        bot.send_message(call.message.chat.id, 'Что будем запоминать?')

@bot.message_handler(content_types=['text'])

def get_message(message):
#'''Функция получения сообщения от пользователя'''
    task = message.text
    chat_id = message.chat.id
    tasks[chat_id] = task
    answer = f'{str(message.chat.first_name)}. По какому номеру позвонить?'
    bot.send_message(message.chat.id, text=answer)
    bot.register_next_step_handler(message, get_number)


def get_number(message):
# Проверка номера и получ    
    phone = message.text
    chat_id = message.chat.id
    print(phone)
    while phone.isdigit() != True or len(str(phone))!=11:
        bot.send_message(message.chat.id, 'Странный номер 🤔, введи новый')
        bot.register_next_step_handler(message, get_number)
        break
    else:
        phones[chat_id] = phone
        answer = f'{str(message.chat.first_name)}. Когда напомнить? (введите время в формате часы:минуты дата.месяц.год)'
        bot.send_message(message.chat.id, text=answer)
        bot.register_next_step_handler(message, get_time)
    
    
def get_time(message):
#'''Функция получения времени задержки от пользователя'''
    timelaps = str(message.text)
    print(timelaps)
    chat_id = message.chat.id
    match = re.fullmatch(r'\d\d:\d\d \d\d.\d\d.\d{4}', timelaps)
    print('YES' if match else 'NO') 
    while  not match:
        bot.send_message(message.chat.id, 'Проверьте правильность ввода 😉')
        bot.register_next_step_handler(message, get_time)
        break    
    else:
        users[chat_id] = get_data(timelaps)
        check_in(message)

def check_in(message):
#'''Функция задержки времени перед отправкой заметки'''
    global i
    chat_id = message.chat.id
    timelaps = users[chat_id]
    task = tasks [chat_id]
    bot.send_message(message.chat.id, text=f'ОК: я напомню сделать {task}, в {timelaps[0]}. Позвоню по номеру {phones[chat_id]}, возьми трубку 😉')
    update_bat(task)
    string = f"schtasks /create /tn My_App_{i} /tr C:\\Users\Sofa\Documents\GitHub\SPBGU-Python-2021\homeworks\Rebrilova\\13.12.21\call.bat /sc once " + timelaps[1]
    print(string)
    ret = os.system(string)
    print(ret)
    i+=1
bot.polling(none_stop=True, timeout=20)

79112871146
14:00 17.12.2021
YES
schtasks /create /tn My_App_4 /tr C:\Users\Sofa\Documents\GitHub\SPBGU-Python-2021\homeworks\Rebrilova\13.12.21\call.bat /sc once /sd 17/12/2021 /st 14:00
0
